# Neural Ordinary Differential Equations (DiffEqFlux tutorial)
https://docs.sciml.ai/DiffEqFlux/stable/examples/neural_ode/

A neural ODE is an ODE where a neural network defines its derivative function. For example, with the multilayer perceptron neural network Lux.Chain(Lux.Dense(2, 50, tanh), Lux.Dense(50, 2)), we can define a differential equation which is u' = NN(u). This is done simply by the NeuralODE struct. Let's take a look at an example.

In [ ]:
using ComponentArrays, Lux, DiffEqFlux, OrdinaryDiffEq, Optimization, OptimizationOptimJL
using OptimizationOptimisers, Random, Plots

In [ ]:
using IRKGaussLegendre

Let's get a time series array from a spiral ODE to train against.

In [ ]:
rng = Xoshiro(0)
u0 = Float32[2.0; 0.0]
datasize = 30
tspan = (0.0f0, 1.5f0)
tsteps = range(tspan[1], tspan[2]; length = datasize)

function trueODEfunc(du, u, p, t)
    true_A = [-0.1 2.0; -2.0 -0.1]
    du .= ((u .^ 3)'true_A)'
end

prob_trueode = ODEProblem(trueODEfunc, u0, tspan)
#ode_data = Array(solve(prob_trueode, Tsit5(); saveat = tsteps))
ode_data = Array(solve(prob_trueode, IRKGL16(); dt=0.25, saveat = tsteps))

Now let's define a neural network with a NeuralODE layer. First, we define the layer. Here we're going to use Lux.Chain, which is a suitable neural network structure for NeuralODEs with separate handling of state variables:

In our model, we used the x -> x.^3 assumption in the model. By incorporating structure into our equations, we can reduce the required size and training time for the neural network, but a good guess needs to be known!

In [ ]:
dudt2 = Chain(x -> x .^ 3, Dense(2, 50, tanh), Dense(50, 2))
p, st = Lux.setup(rng, dudt2)
#prob_neuralode = NeuralODE(dudt2, tspan, Tsit5(); saveat = tsteps)
prob_neuralode = NeuralODE(dudt2, tspan, IRKGL16(); dt=0.25, saveat = tsteps, sensealg = BacksolveAdjoint())

From here we build a loss function around it. The NeuralODE has an optional second argument for new parameters, which we will use to change the neural network iteratively in our training loop. We will use the L2 loss of the network's output against the time series data:

In [ ]:
function predict_neuralode(p)
    Array(prob_neuralode(u0, p, st)[1])
end

function loss_neuralode(p)
    pred = predict_neuralode(p)
    loss = sum(abs2, ode_data .- pred)
    return loss
end

We define a callback function. In this example, we set doplot=false because otherwise it would show every step and overflow the documentation, but for your use case set doplot=true to see a live animation of the training process!

In [ ]:
# Do not plot by default for the documentation
# Users should change doplot=true to see the plots callbacks
function callback(state, l; doplot = false)
    println(l)
    # plot current prediction against data
    if doplot
        pred = predict_neuralode(state.u)
        plt = scatter(tsteps, ode_data[1, :]; label = "data")
        scatter!(plt, tsteps, pred[1, :]; label = "prediction")
        display(plot(plt))
    end
    return false
end

pinit = ComponentArray(p)
callback((; u = pinit), loss_neuralode(pinit); doplot = true)

We then train the neural network to learn the ODE.

Here we showcase starting the optimization with Adam to more quickly find a minimum, and then honing in on the minimum by using LBFGS. By using the two together, we can fit the neural ODE in 9 seconds! (Note, the timing commented out the plotting). You can easily incorporate the procedure below to set up custom optimization problems. For more information on the usage of Optimization.jl, please consult this documentation.

The x and p variables in the optimization function are different from x and p above. The optimization function runs over the space of parameters of the original problem, so x_optimization == p_original.

In [ ]:
# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction((x, p) -> loss_neuralode(x), adtype)
optprob = Optimization.OptimizationProblem(optf, pinit)

result_neuralode = Optimization.solve(
    optprob, OptimizationOptimisers.Adam(0.05); callback = callback, maxiters = 300)

callback((; u = result_neuralode.u), loss_neuralode(result_neuralode.u); doplot = true)

optprob2 = remake(optprob; u0 = result_neuralode.u)

In [ ]:
callback((; u = result_neuralode.u), loss_neuralode(result_neuralode.u); doplot = true)

We then complete the training using a different optimizer, starting from where Adam stopped. We do allow_f_increases=false to make the optimization automatically halt when near the minimum.

In [ ]:
result_neuralode2 = Optimization.solve(
    optprob2, Optim.BFGS(; initial_stepnorm = 0.01); callback, allow_f_increases = false)

callback((; u = result_neuralode2.u), loss_neuralode(result_neuralode2.u); doplot = true)